In [1]:
import nibabel as nib
import numpy as np

In [2]:
# Extracted information from original nifti files. 
# DeepReg throws away this information and doesn't acurately calculate mTRE

case2_affine = np.array([[ 3.33370864e-01,  1.59845248e-01,  3.33878189e-01, -3.39173775e+01],
       [-1.82626724e-01,  4.61422384e-01, -3.86130475e-02, 4.38701019e+01],
       [-3.21150362e-01, -9.64666754e-02,  3.68540883e-01, 6.26287537e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, 1.00000000e+00]])

case21_affine = np.array([[4.00647670e-01, -1.04107566e-01,  2.77501583e-01, -3.58118629e+01],
       [ 1.67786613e-01,  4.64567333e-01, -6.78447485e-02, -8.16707916e+01],
       [-2.44675279e-01,  1.48105368e-01,  4.07874972e-01, -8.83094406e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, 1.00000000e+00]])

case23_affine = np.array([[  0.31273046,   0.25455737,   0.29349992, -41.70222473],
       [ -0.28844342,   0.40450525,  -0.04210297,   2.00646901],
       [ -0.2584393 ,  -0.14343421,   0.40214738,   4.35490799],
       [  0.        ,   0.        ,   0.        ,   1.        ]])

DeepReg_rescale = (128, 128, 128)

case2_size = (128, 161, 106)
case2_scale = np.array([case2_size[0] / DeepReg_rescale[0], 
                        case2_size[1] / DeepReg_rescale[1], 
                        case2_size[2] / DeepReg_rescale[2]])

case21_size = (215, 251, 191)
case21_scale = np.array([case21_size[0] / DeepReg_rescale[0], 
                         case21_size[1] / DeepReg_rescale[1], 
                         case21_size[2] / DeepReg_rescale[2]])

case23_size = (106, 147, 134)
case23_scale = np.array([case23_size[0] / DeepReg_rescale[0], 
                         case23_size[1] / DeepReg_rescale[1], 
                         case23_size[2] / DeepReg_rescale[2]])

In [3]:
def extract_centroid(image):
    """
    Extract centroid from nifti images with landmark spheres
    which have integer values according to labels
    Adapted from: https://gist.github.com/mattiaspaul/f4183f525b1cbc65e71ad23298d6436e

    :param image:
        - shape: (dim_1, dim_2, dim_3) or (batch, dim_1, dim_2, dim_3)
        - tensor or numpy array

    :return positions:
        - numpy array of labels 1
    """
    assert len(image.shape) == 3

    x = np.linspace(0, image.shape[0] - 1, image.shape[0])
    y = np.linspace(0, image.shape[1] - 1, image.shape[1])
    z = np.linspace(0, image.shape[2] - 1, image.shape[2])
    yv, xv, zv = np.meshgrid(y, x, z)
    unique = np.unique(image)[1:]  # don't include 0
    positions = np.zeros((len(unique), 3))
    for i in range(len(unique)):
        label = (image == unique[i]).astype('float32')
        xc = np.sum(label * xv) / np.sum(label)
        yc = np.sum(label * yv) / np.sum(label)
        zc = np.sum(label * zv) / np.sum(label)
        positions[i, 0] = xc
        positions[i, 1] = yc
        positions[i, 2] = zc
    return positions

In [4]:
def calculate_mTRE(xyz_true, xyz_predict):
    assert xyz_true.shape == xyz_predict.shape
    TRE = np.sqrt(np.sum(np.power(xyz_true - xyz_predict, 2), axis=1))
    mTRE = np.mean(TRE)
    return mTRE

In [5]:
def case_TREs(pred_dir, pair_number, num_labels, affine, scale):
    TREs = np.zeros(num_labels)
    for i in range(num_labels):
        label = nib.load(pred_dir + f"pair_{pair_number}/label_{i}/fixed_label.nii.gz")
        pred_label = nib.load(pred_dir + f"pair_{pair_number}/label_{i}/pred_fixed_label.nii.gz")

        label_np = label.get_fdata()
        pred_label_np = pred_label.get_fdata()
        
        label_point = nib.affines.apply_affine(affine, extract_centroid(np.round(label_np))*scale)
        pred_point = nib.affines.apply_affine(affine, extract_centroid(np.round(pred_label_np))*scale)
        
        TREs[i] = calculate_mTRE(label_point, pred_point)
        
    return TREs

# Calulating the mTRE for the 3 test cases for the Deep-Learning Approach

In [6]:
# folder path to the prediction data
prediction_dir = "logs/91_final_test/test/"

## Case 2

In [7]:
case2_TREs = case_TREs(prediction_dir, 0, 15, case2_affine, case2_scale)
case2_TREs

array([4.99592999, 4.7669891 , 4.70059672, 5.15337782, 4.73786057,
       5.40849391, 3.02772649, 5.58445234, 7.1831767 , 5.56912614,
       4.25947545, 6.84398992, 6.75084262, 9.19574115, 4.36709064])

In [8]:
case2_mTRE = np.mean(case2_TREs)
print(f"The mTRE for case 2 was {case2_mTRE}")

The mTRE for case 2 was 5.502991303450743


## Case 21

In [9]:
case21_TREs = case_TREs(prediction_dir, 1, 16, case21_affine, case21_scale)
case21_TREs

array([6.47045722, 3.97960624, 5.02442724, 3.82973097, 4.88126274,
       4.72840392, 4.41295867, 5.18989909, 5.09077191, 3.45221441,
       5.10201935, 4.50546756, 3.7981509 , 4.03465234, 4.06756056,
       3.96497586])

In [10]:
case21_mTRE = np.mean(case21_TREs)
print(f"The mTRE for case 21 was {case21_mTRE}")

The mTRE for case 21 was 4.53328493684857


## Case 23

In [11]:
case23_TREs = case_TREs(prediction_dir, 2, 15, case23_affine, case23_scale)
case23_TREs

array([6.42956974, 6.88552306, 8.1353113 , 6.41557738, 7.54661464,
       8.21287229, 6.65678675, 8.3064462 , 8.11085698, 7.89990213,
       5.21412917, 5.92343757, 5.29677218, 5.93879175, 8.2935333 ])

In [12]:
case23_mTRE = np.mean(case23_TREs)
print(f"The mTRE for case 23 was {case23_mTRE}")

The mTRE for case 23 was 7.017741627767024


###########################################################################################################################

# CNN + SCC Calculations

## mTREs for CNN + SSC (rigid only...non-rigid was just as bad)

In [13]:
SSC_affine_2 = np.array([[0.98227, 0.158062, 0.0692554, 4.02553e-07], 
                         [-0.19717, 0.742934, 0.0995561, 3.75881e-07],
                         [-0.0975693,  -0.0915535, 0.97393, 4.33928e-07],
                         [24.5656, 11.6949, -3.29517, 0.999908]]).T

SSC_affine_21 = np.array([[0.999525, -0.00835235, 0.0296643, 0],
                        [0.00760699, 0.999655, 0.0251491, 0],
                        [-0.0298639, -0.0249113, 0.999244, 0],
                        [1.24686, 1.07985, -1.03928, 1]]).T

SSC_affine_23 = np.array([[0.998656, 0.0435814, 0.0280372, 0],
                         [-0.0420885, 0.99777, -0.0518019, 0],
                         [-0.0302323, 0.0505523, 0.998264, 0],
                         [16.8089, -15.209, 17.1418, 1]]).T

In [14]:
def case_TREs_SSC(pred_dir, pair_number, num_labels, affine, SSC_affine, scale):
    TREs = np.zeros(num_labels)
    for i in range(num_labels):
        label = nib.load(pred_dir + f"pair_{pair_number}/label_{i}/fixed_label.nii.gz")
        pred_label = nib.load(pred_dir + f"pair_{pair_number}/label_{i}/pred_fixed_label.nii.gz")

        label_np = label.get_fdata()
        pred_label_np = pred_label.get_fdata()
        
        transformed_pred =  nib.affines.apply_affine(SSC_affine, extract_centroid(np.round(pred_label_np)))
        
        label_point = nib.affines.apply_affine(affine, extract_centroid(np.round(label_np))*scale)
        pred_point = nib.affines.apply_affine(affine, transformed_pred*scale)
        
        TREs[i] = calculate_mTRE(label_point, pred_point)
        
    return TREs

In [15]:
# folder path to the prediction data
prediction_dir = "logs/91_final_test/test/"

### Case 2

In [16]:
case2_TREs = case_TREs_SSC(prediction_dir, 0, 15, case2_affine, SSC_affine_2, case2_scale)
case2_TREs

array([ 6.89919763,  9.8596831 ,  7.00554537,  6.85536062,  9.31029165,
       11.71223314,  8.48062598,  8.93501441, 12.50851004, 11.91194414,
        8.19810729,  9.4278252 , 11.58480622, 13.08085049,  6.82395628])

In [17]:
case2_mTRE = np.mean(case2_TREs)
print(f"The mTRE for case 2 was {case2_mTRE}")

The mTRE for case 2 was 9.506263436275505


### Case 21

In [18]:
case21_TREs = case_TREs_SSC(prediction_dir, 1, 16, case21_affine, SSC_affine_21, case21_scale)
case21_TREs

array([7.40616255, 5.88861126, 6.73674602, 5.60125225, 6.48628728,
       7.05890398, 7.06025771, 8.22671691, 7.34821473, 5.5905973 ,
       6.84385844, 7.6735398 , 6.07672326, 6.38373297, 6.6709049 ,
       6.39431789])

In [19]:
case21_mTRE = np.mean(case21_TREs)
print(f"The mTRE for case 21 was {case21_mTRE}")

The mTRE for case 21 was 6.715426702991234


### Case 23

In [20]:
case23_TREs = case_TREs_SSC(prediction_dir, 2, 15, case23_affine, SSC_affine_23, case23_scale)
case23_TREs

array([16.66230843, 17.00998231, 18.73673049, 16.30623592, 17.78318241,
       19.03787712, 16.82952526, 18.17905463, 18.74084351, 17.22146913,
       13.56712936, 15.08025951, 14.65098937, 14.79147594, 18.82502464])

In [21]:
case23_mTRE = np.mean(case23_TREs)
print(f"The mTRE for case 23 was {case23_mTRE}")

The mTRE for case 23 was 16.89480586952979


## Sanity Check: moving and fixed images (unscaled)

In [22]:
unscaled_SSC_affine_2 = np.array([[0.991165, 0.0244466, -0.130365, 0],
                                  [-0.00110314, 0.984353, 0.176204, 0],
                                  [0.132633, -0.174503, 0.975683, 0],
                                  [-31.7493, -20.5851, -20.3543, 1]]).T

In [33]:
def case_TREs_SSC_unscaled(pred_dir, pair_number, num_labels, affine, SSC_affine, scale):
    TREs = np.zeros(num_labels)
    for i in range(num_labels):
        label = nib.load(pred_dir + f"pair_{pair_number}/label_{i}/fixed_label.nii.gz")
        pred_label = nib.load(pred_dir + f"pair_{pair_number}/label_{i}/moving_label.nii.gz")

        label_np = label.get_fdata()
        pred_label_np = pred_label.get_fdata() 
        
        transformed_pred =  nib.affines.apply_affine(SSC_affine, extract_centroid(np.round(pred_label_np))*scale)
        
        label_point = nib.affines.apply_affine(affine, extract_centroid(np.round(label_np))*scale)
        pred_point = nib.affines.apply_affine(affine, transformed_pred)
        
        TREs[i] = calculate_mTRE(label_point, pred_point)
        
    return TREs

In [34]:
# folder path to the prediction data
prediction_dir = "logs/91_final_test/test/"

### Case 2

In [35]:
case2_TREs = case_TREs_SSC_unscaled(prediction_dir, 0, 15, case2_affine, unscaled_SSC_affine_2, case2_scale)
case2_TREs

array([21.93468575, 22.79844001, 23.43421962, 23.0585836 , 23.63410866,
       24.40299644, 21.92541149, 24.07677824, 25.79637311, 24.8816825 ,
       22.33319162, 23.11547139, 24.61418481, 23.73617618, 22.96654215])

In [36]:
case2_mTRE = np.mean(case2_TREs)
print(f"The mTRE for case 2 was {case2_mTRE}")

The mTRE for case 2 was 23.5139230384455


This fails the sanity check and something is clearly seems wrong then with how we are testing with CNN+SCC

## Testing just moving and fixed (CNN scaled to 128x128x128)

In [37]:
# folder path to the prediction data
prediction_dir = "logs/91_final_test/test/"

In [38]:
test_affine_SSC_1 = np.array([[0.970629, 0.123898, 0.0611337, 4.38563e-06], 
                          [-0.187429, 0.76746, 0.160733, 3.77978e-06], 
                          [-0.164659, 0.0286062, 1.00569, 4.14213e-06], 
                          [35.0873, 6.22194, -9.00875, 0.999164]]).T

test_affine_SSC_2 = np.array([[0.454008, 0.116395, 0.181164, 1.03077e-06],
                              [-0.18823, 0.51896, -0.371127, 1.94371e-06],
                              [-0.0139738, 0.805056, 1.52682, 1.32621e-06],
                              [31.9663, 2.82907, 33.3063, 0.999684]]).T

test_affine_SSC_3 = np.array([[0.840965, -0.516923, -0.606431, 1.46757e-06],
                              [-0.067257, 0.0511995, -0.291972, 1.07902e-06],
                              [-0.201934, 0.657801, 0.977594, 1.70895e-07],
                              [4.65295, 78.8709, 78.4798, 0.999825]]).T


test_affine_SSC_4 = np.array([[0.998762, -0.00312604, 0.0496527, 0],
                              [0.00306486, 0.999994, 0.00130417, 0],
                              [-0.0496564, -0.00115036, 0.998766, 0],
                              [17.9524, 0.319942, 14.8488, 1]]).T

In [39]:
def case_TREs_SSC__(pred_dir, pair_number, num_labels, affine, SSC_affine, scale):
    TREs = np.zeros(num_labels)
    for i in range(num_labels):
        label = nib.load(pred_dir + f"pair_{pair_number}/label_{i}/fixed_label.nii.gz")
        pred_label = nib.load(pred_dir + f"pair_{pair_number}/label_{i}/moving_label.nii.gz")

        label_np = label.get_fdata()
        pred_label_np = pred_label.get_fdata()
        
        transformed_pred =  nib.affines.apply_affine(SSC_affine, extract_centroid(np.round(pred_label_np)))
        
        label_point = nib.affines.apply_affine(affine, extract_centroid(np.round(label_np))*scale)
        pred_point = nib.affines.apply_affine(affine, transformed_pred*scale)
        
        TREs[i] = calculate_mTRE(label_point, pred_point)
        
    return TREs

In [40]:
case2_TREs = case_TREs_SSC__(prediction_dir, 0, 15, case2_affine, test_affine_SSC_1, case2_scale)
case2_TREs

array([ 8.02437875,  9.50592548,  7.63118018,  7.92777259,  9.52471048,
       10.61070698,  9.18837346,  8.99925713, 10.24778625, 11.19551575,
        8.79173582, 10.3678349 , 10.05218003, 16.41707428,  7.24569682])

In [41]:
case2_mTRE = np.mean(case2_TREs)
print(f"The mTRE for case 2 was {case2_mTRE}")

The mTRE for case 2 was 9.715341926481422


In [42]:
case2_TREs = case_TREs_SSC__(prediction_dir, 0, 15, case2_affine, test_affine_SSC_2, case2_scale)
case2_TREs

array([47.15037785, 26.8587612 , 29.03011669, 50.13070182, 20.44732741,
       25.89887176, 18.86033692, 26.04985593, 37.34920221, 21.7135323 ,
       24.67254931, 38.84744031, 36.18452904, 27.9711532 , 52.92692419])

In [43]:
case2_mTRE = np.mean(case2_TREs)
print(f"The mTRE for case 2 was {case2_mTRE}")

The mTRE for case 2 was 32.27277867452295


In [44]:
case2_TREs = case_TREs_SSC__(prediction_dir, 0, 15, case2_affine, test_affine_SSC_3, case2_scale)
case2_TREs

array([42.02724955, 17.2086385 , 20.22055004, 47.09045049, 17.20753374,
       18.34937172, 17.2592487 , 22.92149937, 31.99936864, 25.17083639,
       15.87855536, 36.59913957, 29.20965219, 39.01910603, 53.18504964])

In [45]:
case2_mTRE = np.mean(case2_TREs)
print(f"The mTRE for case 2 was {case2_mTRE}")

The mTRE for case 2 was 28.88974999421106


In [46]:
case2_TREs = case_TREs_SSC__(prediction_dir, 0, 15, case2_affine, test_affine_SSC_4, case2_scale)
case2_TREs

array([14.01158059, 12.51018908, 12.57980168, 13.91442032, 11.9355573 ,
       11.74945179, 11.39724055, 12.7841462 , 13.32033277,  9.75518109,
       12.67367006, 15.45767684, 13.81351563, 17.12468246, 13.14934387])

In [47]:
case2_mTRE = np.mean(case2_TREs)
print(f"The mTRE for case 2 was {case2_mTRE}")

The mTRE for case 2 was 13.07845268207958
